# Approximate solvers for the Euler equations of gas dynamics

In [Part I](Euler_equations.ipynb) we studied the Riemann problem for Euler equations of inviscid, compressible fluid flow .  As we saw, the exact solution of the Riemann problem is computationally expensive, since it requires solving a set of nonlinear algebraic equations.  In this chapter, we investigate approximate solvers for the Euler equations.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize

import numpy as np
from exact_solvers import Euler
from clawpack import riemann
from utils import riemann_tools
import matplotlib.pyplot as plt
from collections import namedtuple
from ipywidgets import interact
from ipywidgets import widgets
import matplotlib
Primitive_State = namedtuple('State', Euler.primitive_variables)
gamma = 1.4
problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0

## Roe solver
To derive a Roe solver for the Euler equations, we proceed as we did for the shallow water equations, by linearizing about a specially chosen average state.  As in the shallow water case, the Roe linearization for the Euler equations is chosen so that if $(q_l, q_r)$ are related by a single shock, the Roe solver gives the exact solution.

First, recall the one-dimensional Euler eqations of inviscid, compressible flow:

\begin{align}
    \rho_t + (\rho u)_x & = 0 \\
    (\rho u)_t + (\rho u^2 + p)_x & = 0 \\
    E_t + (u(E+p)) = 0.
\end{align}

For an ideal gas, the total energy and pressure are related by the equation of state

$$E = \frac{p}{\gamma-1} + \frac{1}{2}\rho u^2.$$

Our linearization takes the form $q_t + f'(\hat{q})q_x$, where the flux Jacobian is

\begin{align}
    f'(\hat{q}) = \begin{bmatrix} 0 & 1 & 0 \\
                                \frac{1}{2}(\gamma-3)u^2 & (3-\gamma)u & \gamma-1 \\
                                \frac{1}{2}(\gamma-1)u^3-uH & H-(\gamma-1)u^2 & \gamma u \end{bmatrix}.
\end{align}

In [ ]:
solver = riemann.euler_1D_py.euler_roe_1D
num_eqn = riemann.euler_1D_py.num_eqn

problem_data['efix'] = False

print("Roe solver solution to Euler equations:")
states, s, roe_eval = riemann_tools.riemann_solution(solver,
                                    left_state,right_state,
                                    problem_data=problem_data,verbose=True)

#### Phase plane plots

Need to improve the phase plane plots for Euler equations with 3 variables.  Below is a first attempt...

In [ ]:
fig, ax = plt.subplots(1,2,figsize=figsize)
riemann_tools.plot_phase(states,0,1,ax[0])
riemann_tools.plot_phase(states,0,2,ax[1])
riemann_tools.plot_phase_3d(states)

In [ ]:
plot_function = riemann_tools.make_plot_function(states,s,roe_eval,
                              variable_names=Euler.conserved_variables)
interact(plot_function, 
         t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## HLLE Solver

The HLLE solver uses only two waves with a constant state between that is uniquely defined by conservation for any choice of the two wave speeds.  The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characterstic speed $\lambda^3$ in the right state $q_r$.

In [ ]:
solver = riemann.euler_1D_py.euler_hll_1D

# Sod shock tube
left_state  = np.array(Primitive_State(Density = 3.,
                                       Velocity = -0.5,
                                       Pressure = 2.))
right_state = np.array(Primitive_State(Density = 1.,
                                       Velocity = 0.,
                                       Pressure = 1.))

print("HLL solver solution to Euler equations:")
states_hll, s_hll, hll_eval = riemann_tools.riemann_solution(solver,
                                            left_state,right_state,
                                            problem_data=problem_data,
                                            verbose=True)


#### Phase plane plots

Need to improve the phase plane plots for Euler equations with 3 variables.  Below is a first attempt...

In [ ]:
fig, ax = plt.subplots(1,3,figsize=figsize)
riemann_tools.plot_phase(states_hll,0,1,ax[0])
riemann_tools.plot_phase(states_hll,0,2,ax[1])
riemann_tools.plot_phase(states_hll,1,2,ax[2])
riemann_tools.plot_phase_3d(states_hll)

In [ ]:
plot_function = riemann_tools.make_plot_function(states_hll,s_hll,
                              hll_eval, layout='vertical', 
                              variable_names=Euler.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1));

## Comparison of two approximate solvers with the exact solution


In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(left_state ,right_state, gamma)

plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll,states],
                                     [ex_speeds,s_hll,s],
                                     [reval,hll_eval,roe_eval],
                                     [ex_wave_types,['contact']*2,['contact']*3],
                                     ['Exact','HLLE','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.conserved_variables)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

Notice that the Roe solver significantly understimates the shock speed, and even propagates the contact discontinuity in the wrong direction.  Nevertheless, when used as an ingredient in a numerical method, it gives good results.